In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
from pickle import dump, load
from sklearn.model_selection import StratifiedKFold

In [2]:
import torch.utils.data as utils
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR, LambdaLR
from sklearn import metrics
from collections import deque
import copy
import os

In [6]:
INPUT_PATH = Path("/hdd2/kaggle/g2net/input")

In [7]:
df = pd.read_csv(INPUT_PATH / "training_labels.csv")
print(df.shape)
df.head()

(560000, 2)


,id,target
0,00000e74ad,1
1,00001f4945,0
2,0000661522,0
3,00007a006a,0
4,0000a38978,1


In [20]:
files = list((INPUT_PATH / "train").rglob("*.npy"))

In [5]:
# np.random.seed(69)
# sample = np.random.choice(files, 100000, replace=False)
# sample_ar = []

# for s in tqdm(sample):
#     sample_ar.append(np.load(s).T)
# sample_ar = np.concatenate(sample_ar)

100%|██████████| 100000/100000 [15:20<00:00, 108.61it/s]


In [6]:
# scaler = MinMaxScaler(feature_range=(-1, 1))
# scaler.fit(sample_ar)
# sample_ar = scaler.fit_transform(sample_ar)

MinMaxScaler(feature_range=(-1, 1))

In [10]:
# dump(scaler, open('scaler.pkl', 'wb'))

In [13]:
x_sample = np.load("/hdd2/kaggle/g2net/input/train/0/0/0/00000e74ad.npy")
x_sample.shape

(3, 4096)

In [34]:
class Wave_Block_true(nn.Module):
    
    def __init__(self,in_channels,out_channels,dilation_rates):
        super(Wave_Block_true,self).__init__()
        self.num_rates = dilation_rates
        self.convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        self.dil_convs = nn.ModuleList()
        
        self.convs.append(nn.Conv1d(in_channels,out_channels,kernel_size=1))
        dilation_rates = [2**i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            self.dil_convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=3,padding=dilation_rate, dilation=dilation_rate))
            self.filter_convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=3,padding=1))
            self.gate_convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=3,padding=1))
            self.convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=1))
            
        self.end_block = nn.Sequential(nn.ReLU(), nn.Conv1d(out_channels,out_channels,kernel_size=1), nn.ReLU(), nn.Conv1d(out_channels,out_channels,kernel_size=1))
            
    def forward(self,x):
        x = self.convs[0](x)
#         res = x
        skip = 0
        for i in range(self.num_rates):
            
            res = x
            x = self.dil_convs[i](x)
            x = torch.mul(torch.tanh(self.filter_convs[i](x)), torch.sigmoid(self.gate_convs[i](x))) 
            x = self.convs[i+1](x)
            skip = skip + x
            #x += res
            x = x + res
        
        x = self.end_block(skip)
        return x

class Andrewnet_v3_true(nn.Module):

    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels
        
        self.first_conv = nn.Sequential(nn.Conv1d(in_channels,64,7,padding=3), nn.BatchNorm1d(64), nn.ReLU())
        self.waveblock_1 = nn.Sequential(Wave_Block_true(64, 16, 12), nn.BatchNorm1d(16))
        self.waveblock_2 = nn.Sequential(Wave_Block_true(16, 32, 8), nn.BatchNorm1d(32))
        self.waveblock_3 = nn.Sequential(Wave_Block_true(32, 64, 4), nn.BatchNorm1d(64))
        self.waveblock_4 = nn.Sequential(Wave_Block_true(64, 128, 1), nn.BatchNorm1d(128))
        self.waveblock_5 = nn.Sequential( 
                                        nn.Conv1d(128,128,7,padding=3), 
                                        nn.BatchNorm1d(128), 
                                        nn.ReLU())

        self.dropout = nn.Dropout(p=0.2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128,1)
        
        
    def forward(self, x):
        x = self.first_conv(x)
        x = self.waveblock_1(x)
        x = self.waveblock_2(x)
        x = self.waveblock_3(x)
        x = self.waveblock_4(x)
        x = self.waveblock_5(x)
        x = self.dropout(x)
        x = self.avgpool(x)
        x = self.fc(x.view(-1, 128))
        return x

In [40]:
# x = torch.tensor(x_sample).unsqueeze(0).float()
# model = Andrewnet_v3_true(3)
# pred = model(x)
# print(pred.shape)

torch.Size([1, 1])


,id,target,fold
0,00000e74ad,1,0
1,00001f4945,0,0
2,0000661522,0,4
3,00007a006a,0,3
4,0000a38978,1,0
...,...,...,...
559995,ffff9a5645,1,0
559996,ffffab0c27,0,2
559997,ffffcf161a,1,2
559998,ffffd2c403,0,4


In [22]:
FILE_PATH_DICT = {x.stem: str(x) for x in files}

In [16]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=69)
df['fold'] = -1
for f, (train_ids, val_ids) in enumerate(skf.split(df.index, y=df['target'])):
    df.loc[val_ids, 'fold'] = f

In [26]:
df["path"] = df["id"].apply(lambda x: FILE_PATH_DICT[x])

In [3]:
class SignalDataset(Dataset):
    def __init__(self, df, scaler):
        self.df = df
        self.scaler = scaler
        
    def __len__(self):     
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "path"]
        target = float(self.df.loc[idx, "target"])
        x = self.scaler.transform(np.load(path).T).T
        return x, target

In [46]:
BS = 64
FOLD = 0

scaler = load(open('scaler.pkl', 'rb'))
train_ds = SignalDataset(df[df['fold']!=FOLD].reset_index(drop=True)[:1000], scaler)
val_ds = SignalDataset(df[df['fold']==FOLD].reset_index(drop=True)[:1000], scaler)

train_loader = utils.DataLoader(train_ds, shuffle=True, num_workers=11, batch_size=BS, pin_memory=True)
val_loader = utils.DataLoader(val_ds, shuffle=False, num_workers=11, batch_size=BS, pin_memory=True)


model = Andrewnet_v3_true(3)

model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.BCEWithLogitsLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='max', verbose=True, patience=5, factor=0.5, eps=1e-12)


best_score = 0
best_models = deque(maxlen=5)
for e in range(100):

    # Training:
    train_loss = []
    model.train()
    
    for x, y in tqdm(train_loader, ncols = 70):   
        optimizer.zero_grad()
        x = x.cuda().float()
        y = y.cuda().float().unsqueeze(1)
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())


    val_loss = []

    val_true = []
    val_pred = []
    model.eval()  
    with torch.no_grad():
        for x, y in tqdm(val_loader, ncols=50):
            x = x.cuda().float()
            y = y.cuda().float().unsqueeze(1)
            pred = model(x)
            loss = loss_fn(pred, y)
            val_loss.append(loss.item())
            
            pred = pred.sigmoid().cpu().data.numpy()
            val_pred.append(pred)
            val_true.append(y.cpu().numpy())
    
    val_loss = np.mean(val_loss)
    
    val_true = np.concatenate(val_true).reshape(-1,)
    val_pred = np.concatenate(val_pred).reshape(-1,)
    
    final_score = metrics.roc_auc_score(val_true, val_pred)
        
    # print(f'Epoch: {e:03d}; lr: {lr:.06f}; train_loss: {np.mean(train_loss):.05f}; val_loss: {val_loss:.05f}; ', end='')
    print(f'Epoch: {e:03d}; train_loss: {np.mean(train_loss):.05f} val_loss: {val_loss:.05f}; roc: {final_score:.5f}', end=' ')
 
    if final_score > best_score:
        best_score = final_score
        torch.save(model.state_dict(), f"baseline_f0.pt")        
    else:
        print()

    # print(metrics.classification_report(val_true, val_pred))
        
    scheduler.step(final_score)

  0%|                                          | 0/16 [00:00<?, ?it/s]

Epoch: 000; val_loss: 0.69408; roc: 0.49936 

  0%|                                          | 0/16 [00:00<?, ?it/s]

Epoch: 001; val_loss: 0.69367; roc: 0.50427 

 44%|██████████████▉                   | 7/16 [00:02<00:02,  3.16it/s]


KeyboardInterrupt: 

In [83]:
class SignalDataset(Dataset):
    def __init__(self, df):
        self.df = df

        
    def __len__(self):     
        return 250#len(self.df)

    def __getitem__(self, idx):
        rand_id = np.random.randint(len(self.df), size=1)[0]
        return df.loc[rand_id, "value"]
    
df = pd.DataFrame()
df["value"] = np.arange(500)

    
train_loader = utils.DataLoader(SignalDataset(df), shuffle=True, num_workers=10, batch_size=10, pin_memory=False)

In [84]:
xx = list(train_loader)

In [85]:
xx

[tensor([272, 394, 436, 385, 134,  98, 118, 133, 371, 295]),
 tensor([ 17, 398, 205, 131, 145, 108, 417,  34, 409, 195]),
 tensor([328, 168, 225, 149,  42, 364, 492, 188, 426, 431]),
 tensor([147, 352,  79, 201,  93, 447, 138, 162,  30, 190]),
 tensor([376, 400,  97,  61, 325,  65, 438, 376, 105, 343]),
 tensor([ 15, 444, 324,  53, 445, 477, 329, 392, 432, 103]),
 tensor([427, 124, 318, 320, 319, 109, 341, 112, 110, 420]),
 tensor([ 89, 443, 142, 377,  65,  14,  58, 236,  78, 330]),
 tensor([ 84, 220, 353, 351, 171, 426, 492, 430, 271, 412]),
 tensor([193, 363, 208, 456, 163,  36, 212, 275, 474, 108]),
 tensor([162,  15,  35, 273,  48, 171,  10, 498, 235, 157]),
 tensor([346, 365, 484, 263, 281, 377, 172, 473, 161, 369]),
 tensor([135, 470,  78,  86, 326, 288, 312, 300, 245,  49]),
 tensor([  7, 111, 183, 274,  43, 200, 105, 295, 167, 343]),
 tensor([138,  26, 388, 186, 311, 183, 128, 303,  71, 200]),
 tensor([234,   0, 219, 474, 230, 328, 168, 220, 236, 340]),
 tensor([114, 221, 447, 